In [1]:
import pyrosetta
from pyrosetta.rosetta.protocols.carbohydrates import SimpleGlycosylateMover
from pyrosetta.rosetta.core.pose.carbohydrates import glycosylate_pose
from pyrosetta.rosetta.core.pack.task.operation import (
    InitializeFromCommandline,
    RestrictToRepacking,
)
from pyrosetta.rosetta.core.select.residue_selector import (
    ResidueIndexSelector,
    NeighborhoodResidueSelector,
)
from pyrosetta.rosetta.core.simple_metrics.metrics import RMSDMetric
from pyrosetta.rosetta.core.pack.task.operation import (
    OperateOnResidueSubset,
    PreventRepackingRLT,
)
from pyrosetta.rosetta.protocols.rigid import (
    RigidBodyPerturbNoCenterMover as RBPNCMover,
)
from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.core.select.movemap import MoveMapFactory

## Full imports of libraries 
from pyrosetta.rosetta.core.scoring.constraints import *
from pyrosetta.rosetta.protocols.carbohydrates import *
from pyrosetta.rosetta.core.scoring import *

## ***************************************************************
from pyrosetta.rosetta.core.select.residue_selector import (
    AndResidueSelector, 
    NeighborhoodResidueSelector, 
    NotResidueSelector,
    ChainSelector
)

from pyrosetta.rosetta.core.pack.task import TaskFactory

from pyrosetta.rosetta.protocols.denovo_design.movers import FastDesign

from pyrosetta.rosetta.core.pack.task.operation import (
    PreventRepackingRLT, 
    OperateOnResidueSubset, 
    RestrictToRepackingRLT,
    IncludeCurrent
)

In [2]:
#from pyrosetta.rosetta.core.pack.task.

In [3]:
pyrosetta.init("-include_sugars -lock_rings -maintain_links -auto_detect_glycan_connections -alternate_3_letter_codes pdb_sugar -auto_setup_metals -constraints:cst_fa_file starting_structure/6N1B_catalytic_and_specific.cst")

PyRosetta-4 2020 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release 2020.40+release.f01fa77e7385c2c0bdac6e346b24a6d8d900e7c7 2020-10-02T11:47:11] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python37.Release r267 2020.40+release.f01fa77 f01fa77e7385c2c0bdac6e346b24a6d8d900e7c7 http://www.pyrosetta.org 2020-10-02T11:47:11
core.init: {0} command: PyRosetta -include_sugars -lock_rings -maintain_links -auto_detect_glycan_connections -alternate_3_letter_codes pdb_sugar -auto_setup_metals -constraints:cst_fa_file starting_structure/6N1B_catalytic_and_specific.cst -database /home/souvadra/anaconda3/envs/pyrosetta_env/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG dev

In [4]:
!ls starting_structure/

6N1B-A_antigen.pdb  6N1B_catalytic_and_specific.cst


In [5]:
pose = pyrosetta.pose_from_pdb("starting_structure/6N1B-A_antigen.pdb")

core.chemical.GlobalResidueTypeSet: {0} Finished initializing fa_standard residue type set.  Created 1641 residue types
core.chemical.GlobalResidueTypeSet: {0} Total time to initialize 1.32539 seconds.
core.import_pose.import_pose: {0} File 'starting_structure/6N1B-A_antigen.pdb' automatically determined to be of type PDB
core.io.util: {0} Automatic glycan connection is activated.
core.io.util: {0} Start reordering residues.
core.io.util: {0} Corrected glycan residue order (internal numbering): [390, 391, 392]
core.io.util: {0} 
core.io.pose_from_sfr.PoseFromSFRBuilder: {0} Setting chain termination for 391
core.io.pose_from_sfr.PoseFromSFRBuilder: {0} CA 386 was added by a jump, with base residue 94
core.io.pose_from_sfr.PoseFromSFRBuilder: {0} CA 387 was added by a jump, with base residue 96
core.io.pose_from_sfr.PoseFromSFRBuilder: {0} CA 388 was added by a jump, with base residue 230
core.io.pose_from_sfr.PoseFromSFRBuilder: {0} CA 389 was added by a jump, with base residue 234
cor

In [ ]:
sfxn = pyrosetta.create_score_function("ref2015_cart.wts")
inital_energy = sfxn(pose)

sfxn.set_weight(atom_pair_constraint, 1.0)
add_fa_constraints_from_cmdline(pose, sfxn)

In [ ]:
sfxn.show(pose)

In [ ]:
print(pose.sequence())
print("Length of the protein+sugar is:", len(pose.sequence()))

In [ ]:
aa_res = ChainSelector("A")
sugar_res = NotResidueSelector(ChainSelector("A"))

In [ ]:
nbr_selector = NeighborhoodResidueSelector(sugar_res, 8, True)

not_to_design = AndResidueSelector(sugar_res, nbr_selector)
only_to_design = NotResidueSelector(not_to_design)

In [ ]:
disable_packing = PreventRepackingRLT()
repack_only = RestrictToRepackingRLT()

In [ ]:
repack_nbr = OperateOnResidueSubset(
    repack_only, nbr_selector, flip_subset=False
)
nopack_sugar = OperateOnResidueSubset(
    disable_packing, sugar_res, flip_subset=False
)

In [ ]:
tf = TaskFactory()
tf.push_back(InitializeFromCommandline())
tf.push_back(IncludeCurrent())
tf.push_back(repack_nbr)
tf.push_back(nopack_sugar)

packer_task = tf.create_task_and_apply_taskoperations(pose)
# View the PackerTask
print(packer_task)

In [ ]:
mmf = MoveMapFactory()
mmf.all_bb(True)
mmf.all_chi(True)
mmf.add_bb_action(
    pyrosetta.rosetta.core.select.movemap.move_map_action.mm_disable,
    only_to_design
)
mm = mmf.create_movemap_from_pose(pose)

In [ ]:
print(mm)

In [ ]:
fd = FastDesign()
fd.set_scorefxn(sfxn)
fd.cartesian(True)
fd.set_task_factory(tf)
fd.set_movemap_factory(mmf)
fd.minimize_bond_angles(True)
fd.minimize_bond_lengths(True)
fd.min_type("lbfgs_armijo_nonmonotone")
fd.max_iter(1)

In [ ]:
fd.apply(pose)